# IMPORTS

In [1]:
# Imports 3third party

import time
import requests
import pandas as pd
import json
import boto3
import io
import boto3
from datetime import datetime, timedelta
import pyarrow as pa
import pyarrow.parquet as pq
from dotenv import load_dotenv

In [5]:
# IMPORTS/variables custom 
import os
import sys
sys.path.append(os.path.abspath('.'))

load_dotenv()
PATH_DIR = os.getenv("path_variable")
PROFILE_NAME = os.getenv("profile_name")

# Get the absolute path
absolute_directory = os.path.abspath(PATH_DIR)

# Add the directory to sys.path
sys.path.append(absolute_directory)


# Imports Resources
import pandas as pd
from pandas import json_normalize
from io import StringIO
from datetime import datetime, timedelta
from api_requests import Brapi_STOCKS_API, Brapi_notSTOCKS__API
#from parquet_model import write_parquet_to_s3_raw
from log_manager import S3LogManager
#from week_days import get_previous_business_days
from util import control_sucess, infer_schema, convert_to_string
from json_upload_local import write_json_to_s3
from stocks_fiis_modules import validate_data_stocks
import time

# Import configs_dev in notebook_dir 
from config_dev import BASE_URL, REGION_NAME, \
    BUCKET, RAW_PREFIX, LOG_PREFIX, NAME_FUN, TOKEN_GENERATOR


In [5]:
# print("CHANGE ALL THE MOBILE VARIABLES IN config_dev.py FILE")
# print(f"{BUCKET} \n{RAW_PREFIX}\n {BASE_URL}")


# _DEV_

In [7]:
##  Dev/_Debugging old version_

# Develop your code here


tickers = ["PETR4", "SUZB3", "MGLU3"]



params = {
    'range': '1d',
    'interval': '1d',
    'fundamental': 'true',
    'token': TOKEN_GENERATOR,
}


api = Brapi_STOCKS_API(
    url=BASE_URL,
    tickers=tickers, 
    params=params
)

data = api.get_quotes()
schema = infer_schema(data)
check_json = validate_data_stocks(data)
timestamp_SP = datetime.now() - timedelta(hours=3)
#write_json_to_s3(check_json, BUCKET, RAW_PREFIX, timestamp_SP, PROFILE_NAME)

In [4]:
# import pprint
# pprint.pprint(check_json)

### Lambda fun model below : Uncode it to dev

In [8]:

# V0 inicial structure for logging ect
def lambda_handler(event, context):
    inicio = time.perf_counter()
    control = "OK"
    timestamp_SP = datetime.now() - timedelta(hours=3)
    timestamp_SP_api_pull = timestamp_SP - timedelta(days=1)
    
    # loop df ... [!WARNING] if dont use it change in row 58
    #df_append = pd.DataFrame()

    # api class for requests
    tickers = ["PETR4", "SUZB3", "MGLU3"]

    params = {
        'range': '1d',
        'interval': '1d',
        'fundamental': 'true',
        'token': TOKEN_GENERATOR,
    }
    
    
    api = Brapi_STOCKS_API(
        url=BASE_URL,
        tickers=tickers, 
        params=params
    )
    
    json_data = api.get_quotes()
    
    # --------------------Api requests wra wrangling
    # ------------------- Use Try and execept
 
    #----------------- execept + control_var for logs management
    try:
        schema =infer_schema(data)
        check_json = validate_data_stocks(json_data)

    #----------------- execept + control_var for logs management

    
    except Exception as e:
         control = f'Error msg: {e}'
         print(
             "Project: The request returned 0 items. Original message: {msg}".format(
                 msg=e
             )
         )
                        
    # -------------------------- usual final strucuture for uploading and logging the ingestion.
    
    sucess_var = control_sucess(control)

    file_s3= 0
    if control == "OK":
        file_s3, file_size = write_json_to_s3(check_json, BUCKET, RAW_PREFIX, timestamp_SP, PROFILE_NAME)
        print(f"Project: Json file saved to S3 at: \n {file_s3}")
        print(f"Project: Json file size: {file_size} bytes")
        
    fim = time.perf_counter()
    log_manager = S3LogManager(BUCKET, LOG_PREFIX, control)
    time_var = round((fim - inicio),2)
    log_manager.add_log_entry(NAME_FUN, sucess_var, time_var, control, file_size)

# ! lambda_handler('','') !


### TESTING / DEBUGING IT

In [9]:
import time
import logging

start_time = time.perf_counter()

try:
    lambda_handler(None, None)
    logging.basicConfig(filename = "scrapper.log", level=logging.INFO, format='%(asctime)s - %(message)s', datefmt = '%Y-%m-%d %H:%M:%S')
    end = time.perf_counter()
    time = end - start_time
    logging.info(f"{NAME_FUN} - success - time {time:.2f}.")

except Exception as e:
    logging.error(f"Error.{NAME_FUN}:{e}")

Project: Json file saved to S3 at: 
 s3://aws-serveless-project-lambda-teste/dev/raw/brapi_api/date_ingestion=20240510/file.json
Project: Json file size: 2451 bytes
